In [1]:
# pip install nltk

In [2]:
# import nltk
# nltk.download('punkt_tab')
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# model = AutoModelForSeq2SeqLM.from_pretrained('eenzeenee/t5-base-korean-summarization')
# tokenizer = AutoTokenizer.from_pretrained('eenzeenee/t5-base-korean-summarization')

# prefix = "summarize: "
# sample = """#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
# #Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
# #Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
# #Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
# #Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
# #Person2#: 알겠습니다.
# #Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
# #Person2#: 네.
# #Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
# #Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
# #Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
# #Person2#: 알겠습니다, 감사합니다, 의사선생님."""

# inputs = [prefix + sample]


# inputs = tokenizer(inputs, max_length=512, truncation=True, return_tensors="pt")
# output = model.generate(**inputs, num_beams=3, do_sample=True, min_length=10, max_length=64)
# decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
# result = nltk.sent_tokenize(decoded_output.strip())[0]

# print('RESULT >>', result)

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from rouge import Rouge
from transformers import EarlyStoppingCallback
from transformers.trainer_callback import TrainerCallback
from transformers import logging

import torch
import gc

from datetime import datetime
import time
from zoneinfo import ZoneInfo

import os
import re

from pathlib import Path

In [2]:
import logging

# 로거 생성
logger = logging.getLogger(__name__)

# 로거 레벨 설정
logger.setLevel(logging.INFO)

# 콘솔 핸들러 생성
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# 포매터 생성
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

# 로거에 핸들러 추가
logger.addHandler(console_handler)

In [ ]:
train_time = datetime.fromtimestamp(time.time(), tz=ZoneInfo("Asia/Seoul")).strftime("%m%d-%H%M%S")
train_time

In [4]:
SPECIAL_TOKENS = ['#Address#', 
                  '#CarNumber#', 
                  '#CardNumber#',
                  '#DateOfBirth#', 
                  '#Email#', 
                  '#PassportNumber#', 
                  '#Person1#', 
                  '#Person2#', 
                  '#Person3#', 
                  '#Person4#', 
                  '#Person5#', 
                  '#Person6#', 
                  '#Person7#',
                  '#PhoneNumber#', 
                  '#SSN#']

In [ ]:
# 토크나이저 및 모델 로드
model_name = 'eenzeenee/t5-base-korean-summarization'
tokenizer = AutoTokenizer.from_pretrained(model_name, additional_special_tokens=SPECIAL_TOKENS)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

In [8]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)
    logger.info(f"Evaluation results: {results}")

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [ ]:
# 데이터 로드
train_df = pd.read_csv('./data/train_cleaned.csv')
dev_df = pd.read_csv('./data/dev.csv')

# 데이터셋 전처리 함수
def preprocess_function(example):
    input_text = f"summarize: {example['dialogue']}"
    model_inputs = tokenizer(input_text, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['summary'], max_length=128, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 데이터셋 전처리
train_dataset = train_df.apply(preprocess_function, axis=1)
dev_dataset = dev_df.apply(preprocess_function, axis=1)

# 학습 인자 설정
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    evaluation_strategy='epoch',
    eval_steps=1,
    save_strategy='epoch',
    save_steps=1,
    logging_strategy='epoch',
    logging_steps=1,
    save_total_limit=5,
    load_best_model_at_end=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # auto_find_batch_size=False,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=4e-05,
    lr_scheduler_type='cosine',
    predict_with_generate=True,
    fp16=True,
    seed=42,
    gradient_accumulation_steps=4,
    # generation_max_length=100,
    do_train=True,
    do_eval=True,
)

# 데이터 콜레이터 설정
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.001,
)

class CUDAMemoryCleanupCallback(TrainerCallback):
    def __init__(self, cleanup_interval=100):
        self.cleanup_interval = cleanup_interval
    
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.cleanup_interval == 0:
            torch.cuda.empty_cache()
            gc.collect()
            # print(f"Step {state.global_step}: CUDA memory cleaned up.")
            # print(f"Current GPU memory usage: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

cuda_cleanup_callback = CUDAMemoryCleanupCallback(cleanup_interval=1)

# Trainer 설정
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics = lambda pred: compute_metrics(tokenizer, pred),
    callbacks=[cuda_cleanup_callback, early_stopping]
)

# 모델 학습
trainer.train()

In [ ]:
# 학습된 모델의 체크포인트 경로
checkpoint_path = './results/checkpoint-9333/'
print(f"checkpoint_path: {checkpoint_path}")

# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)

# Special token 추가
tokenizer.add_special_tokens({'additional_special_tokens': SPECIAL_TOKENS})
model.resize_token_embeddings(len(tokenizer))

# test.csv 파일 읽기
test_data = pd.read_csv('./data/test_cleaned.csv')

# 예측 결과를 저장할 리스트
predictions = []

# 입력 텍스트 전처리
def preprocess_function(text):
    input_text = f"summarize: {text}"
    return tokenizer(input_text, max_length=512, truncation=True, return_tensors='pt')

for dialogue in test_data['dialogue']:
    inputs = preprocess_function({dialogue})
    
    summary_ids = model.generate(
        inputs['input_ids'],
        # num_beams=4,
        # max_length=50,
        # early_stopping=True,
        do_sample=False,
        repetition_penalty = 1.1
    )
    
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=False)
    print(summary)
    predictions.append(summary)

In [6]:
# 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰을 제거합니다.
def remove_default_token(predictions):
    remove_tokens = ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    preprocessed_summary = predictions.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    return preprocessed_summary

preprocessed_summary = remove_default_token(predictions)

In [ ]:
def clean_prediction(text):
    if not isinstance(text, str):
        return text
    
    # 앞뒤 따옴표 제거 (작은따옴표와 큰따옴표 모두 처리)
    text = text.strip("'\"")

    # 문자열의 앞뒤에서 공백 문자(스페이스, 탭, 줄바꿈 등)를 제거
    text = text.strip()
    
    text = text.replace('\r', '')
    text = text.replace('\n', '')

    # 연속된 공백을 단일 공백으로 변경
    text = re.sub(r'\s+', ' ', text)
    
    return text

cleaned_sentences = [clean_prediction(sentence) for sentence in preprocessed_summary]
cleaned_sentences

In [ ]:
test_data['summary'] = cleaned_sentences
test_data.head(50)

In [ ]:
# 결과를 파일로 저장
test_data = test_data.drop('dialogue', axis=1)
test_data.to_csv('test_predictions.csv', index=False)

print("예측이 완료되었습니다. 결과가 'test_predictions.csv' 파일로 저장되었습니다.")